In [10]:
#Library to read input file
import pandas as pd
#For math operations
import numpy as np
# Used for splitting training data into validation and training set
from sklearn.model_selection import train_test_split

#Used for making TF-IDF vector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#used for computing precision_recall_fscore
from sklearn.metrics import precision_recall_fscore_support

#used for computing accuracy
from sklearn.metrics import accuracy_score

#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier

#SVM classifier
from sklearn.svm import SVC

#Compute F1-score
from sklearn.metrics import f1_score

#Compute Precision 
from sklearn.metrics import precision_score

#Computes Recall
from sklearn.metrics import recall_score

# NLTK lib for word tokenization and other pre-processing
from nltk import word_tokenize
import nltk

#Preprocessing libraries
from nltk.corpus import stopwords
from nltk import word_tokenize,WordNetLemmatizer

#regular expression
import re


In [11]:
#Reading Training dataa
data = pd.read_csv('training.csv',header=None, names=['text','character','gender'])

data.head()

 
 

,text,character,gender
0,Someone had fun.,SEAN,male
1,"It's no problem, honestly. Go on, go and open ...",SHIRLEY,female
2,Last night was better than ever. What's all th...,MAX,male
3,Have you checked the answerphone? Any calls?,IAN,male
4,Oscar's asleep.,MAX,male


In [12]:
#PRE-PROCESSING
stoplist = stopwords.words('english')

#lemmatizes 
def lem_message(text):
    temp=[]
    text = text.split()
    wl=WordNetLemmatizer()
    for i in text:
        temp.append(wl.lemmatize(i,pos='v'))
    return " ".join(temp)

#this function removes stopwords
def remove_stopwords(text,stoplist):
    temp=[]
#     print(text)
#     print (len(text))
    text = text.split()
    for i in range (len(text)):
        if text[i] not in stoplist:
            temp.append(text[i])
    return " ".join(temp)

import string
x = string.punctuation
for i in range (data.shape[0]):
    data["text"].iloc[i] = re.sub('['+x+']' ,'',str(data["text"].iloc[i])) #thi removes special chars
    data["text"].iloc[i] = remove_stopwords(data["text"].iloc[i],stoplist)

In [13]:
data.head()

,text,character,gender
0,Someone fun,SEAN,male
1,Its problem honestly Go go open launderette Leave,SHIRLEY,female
2,Last night better ever Whats Anything interesting,MAX,male
3,Have checked answerphone Any calls,IAN,male
4,Oscars asleep,MAX,male


In [14]:
# Getting reading the data for processing Embeddings
corpus = []
sentences = []
from gensim.models import Word2Vec
for i in range (data.shape[0]):
    corpus.extend(str(data["text"].iloc[i]).split())
    sentences.append(str(data["text"].iloc[i]).split())

In [15]:
#Training a Word2Vec model on the data
from gensim.models import Word2Vec
model_W2V = Word2Vec(sentences, size=200, window=10, min_count=1) #initilize
model_W2V.train(sentences, total_examples=len(set(corpus)),epochs=100) #trains model

(5241179, 6239200)

In [16]:
np.unique(data["character"])

array(['BRADLEY', 'CHRISTIAN', 'CLARE', 'GARRY', 'HEATHER', 'IAN', 'JACK',
       'JANE', 'MAX', 'MINTY', 'PHIL', 'RONNIE', 'ROXY', 'SEAN',
       'SHIRLEY', 'STACEY', 'STEVEN', 'TANYA'], dtype=object)

In [17]:
data ["vecs"] = data["text"] #make new column for vecs in dataframe

In [18]:
for i in range(data.shape[0]):
    sent = data["vecs"].iloc[i]
    sent_vec = np.zeros((200,))
    for word in sent:
        if word in set(model_W2V.wv.index2word): #check if words are present in vocab
            sent_vec += model_W2V[word] #adding vectors for words
    
    data["vecs"].iloc[i] = sent_vec
#     break



c:\users\themo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
data.head()

,text,character,gender,vecs
0,Someone fun,SEAN,male,"[0.4833850860595703, -0.07849548012018204, -0...."
1,Its problem honestly Go go open launderette Leave,SHIRLEY,female,"[0.17704612456145696, 0.6082140542566776, 0.30..."
2,Last night better ever Whats Anything interesting,MAX,male,"[2.3692171424627304, 0.0736575685441494, -1.71..."
3,Have checked answerphone Any calls,IAN,male,"[1.7213956512132427, 0.31234126165509224, -1.3..."
4,Oscars asleep,MAX,male,"[-0.22403573025076184, 0.2623397260904312, -0...."


In [20]:
#We had shape issues so we stacked pandas into numpy array
X_stacked = data["vecs"].iloc[0]
for i in range(1,data.shape[0]):
    X_stacked = np.vstack((X_stacked,data["vecs"].iloc[i]))

In [21]:
#Prepares TF_IDF scores for training data
data['text']=data['text'].str.lower()
corpus=data['text'].values.astype('U')
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_tf = vectorizer.fit_transform(corpus) #Tf x value
print(X_tf.shape)

featureNames=vectorizer.get_feature_names()

 

(10113, 42153)


In [22]:
# encode categorical data [classA,classB] -> [0,1]
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
 
labelencoder_Y = LabelEncoder()

Y= labelencoder_Y.fit_transform(data.character)
 
X_train, X_test, y_train, y_test = train_test_split(X_stacked, Y, test_size=0.33, random_state=42)

In [23]:
def evaluate2(y_test,y_pred,algoText='Random Forest (Baseline-1)'):
    print('------------------------------------------------------------------')
    print('Results of  ',algoText)

    print('------------------------------------------------------------------')


    print('Accuracy    ',accuracy_score(y_test, y_pred))

    print('Recall      ',recall_score(y_test, y_pred,average='micro'))
    
    print('Precision   ',precision_score(y_test,y_pred,average='micro'))

    print('F1-Score    ',f1_score(y_test,y_pred,average='micro'))
    
    

In [24]:
def evaluate(y_test,y_pred,algoText='Random Forest (Baseline-1)'):
    print('------------------------------------------------------------------')
    print('Results of  ',algoText)

    print('------------------------------------------------------------------')
    
    print('Accuracy    ',accuracy_score(y_test, y_pred))
    print("Precision - Recall - F1 scores with different averaging")
    print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
    print(precision_recall_fscore_support(y_test, y_pred, average='micro'))
    print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

     

In [25]:
X_train.shape

(6775, 200)

In [26]:
#Training RF on Word2Vec
clfRF = RandomForestClassifier(n_estimators=100, min_samples_split=2)
clfRF.fit(X_train, y_train) 
y_pred=clfRF.predict(X_test)
evaluate(y_test,y_pred,'Random Forest + Word2Vec (Baseline-1) on splitted_test')


------------------------------------------------------------------
Results of   Random Forest + Word2Vec (Baseline-1) on splitted_test
------------------------------------------------------------------
Accuracy     0.11473936488915518
Precision - Recall - F1 scores with different averaging
(0.08024587111423123, 0.07218486373617489, 0.06509679105356693, None)
(0.11473936488915518, 0.11473936488915518, 0.11473936488915518, None)
(0.09282932466744362, 0.11473936488915518, 0.08623735198500111, None)


In [27]:
#Training SVM on Word2vec
clfSVM=SVC(gamma='auto',kernel='linear', C = 1)
clfSVM.fit(X_train, y_train) 
y_pred=clfSVM.predict(X_test)
evaluate(y_test,y_pred,'SVM + Word2vec (Baseline-2) on splitted_test')

print(np.unique(y_pred))


------------------------------------------------------------------
Results of   SVM + Word2vec (Baseline-2) on splitted_test
------------------------------------------------------------------
Accuracy     0.1318154583582984
Precision - Recall - F1 scores with different averaging
(0.0307154783718562, 0.05779820959907018, 0.02133567613196795, None)
(0.1318154583582984, 0.1318154583582984, 0.1318154583582984, None)
(0.048529295525320176, 0.1318154583582984, 0.04304917032479857, None)
[ 5  7  8 10 11 14 15 17]


c:\users\themo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
#TEst train split TF_IDF SCORES
X_train, X_test, y_train, y_test = train_test_split(X_tf, Y, test_size=0.33, random_state=42)

In [29]:
#Training RF with TF_IDF
clfRF_TF = RandomForestClassifier(n_estimators=100, min_samples_split=2)
clfRF_TF.fit(X_train, y_train) 
y_pred=clfRF_TF.predict(X_test)
evaluate(y_test,y_pred,'Random Forest + TF-IDF (Baseline-1) on splitted_test')


------------------------------------------------------------------
Results of   Random Forest + TF-IDF (Baseline-1) on splitted_test
------------------------------------------------------------------
Accuracy     0.18184541641701618
Precision - Recall - F1 scores with different averaging
(0.17788841530751692, 0.14714252747003756, 0.1502546986091796, None)
(0.18184541641701618, 0.18184541641701618, 0.18184541641701618, None)
(0.1819788138583275, 0.18184541641701618, 0.16893981579716685, None)


In [30]:
#Training SVM with TF_IDF
clfSVM_TF=SVC(gamma='auto',kernel='linear', C = 1)
clfSVM_TF.fit(X_train, y_train) 
y_pred=clfSVM_TF.predict(X_test)
evaluate(y_test,y_pred,'SVM + TF-IDF (Baseline-2) on splitted_test')

# print(np.unique(y_pred))


------------------------------------------------------------------
Results of   SVM + TF-IDF (Baseline-2) on splitted_test
------------------------------------------------------------------
Accuracy     0.17525464349910125
Precision - Recall - F1 scores with different averaging
(0.24102439182522697, 0.11452319070047873, 0.10912635788105933, None)
(0.17525464349910125, 0.17525464349910125, 0.17525464349910125, None)
(0.22599015772971534, 0.17525464349910125, 0.13426261954660396, None)


In [31]:
testdata = pd.read_csv('test.csv',header=None, names=['text','character','gender'])

testdata.head()
#PRE-PROCESSING------------------------------
stoplist = stopwords.words('english')
#lemmatizes 
def lem_message(text):
    temp=[]
    text = text.split()
    wl=WordNetLemmatizer()
    for i in text:
        temp.append(wl.lemmatize(i,pos='v'))
    return " ".join(temp)

#this function removes stopwords
def remove_stopwords(text,stoplist):
    temp=[]
#     print(text)
#     print (len(text))
    text = text.split()
    for i in range (len(text)):
        if text[i] not in stoplist:
            temp.append(text[i])
    return " ".join(temp)

import string
x = string.punctuation
for i in range (testdata.shape[0]):
    testdata["text"].iloc[i] = re.sub('['+x+']' ,'',str(testdata["text"].iloc[i])) #thi removes special chars use this only :p for every sentence in train and test
    testdata["text"].iloc[i] = lem_message(str(testdata["text"].iloc[i]))
    testdata["text"].iloc[i] = remove_stopwords(testdata["text"].iloc[i],stoplist)
    
#------------------------------------------------

testData['text']=testData['text'].str.lower()
corpus=testData['text'].values.astype('U')



vectorizer = TfidfVectorizer(vocabulary=featureNames,ngram_range=(1,2))
tX = vectorizer.fit_transform(corpus)
labelencoder_Y = LabelEncoder()

#Y = labelencoder_Y.fit_transform(data.gender)
y_Test= labelencoder_Y.fit_transform(testData.character)

NameError: name 'testData' is not defined

In [ ]:
#Predicting Test
y_Pred=clfRF_TF.predict(tX)
evaluate(y_Test,y_Pred,'Random Forest + TF-IDF (Baseline-1) on Test Dataset') 

In [ ]:
y_Pred=clfSVM_TF.predict(tX)
evaluate(y_Test,y_Pred,'SVM +TF-IDF (Baseline-2) on Test Dataset')

In [ ]:
#This function extracts POS tagging for each word of the sentence and add it with the sentence
def POSBasedFeatures(data):
    print(len(data))
    taggedData=[]
    for row in data:
        
        d=''
        #print('...')
        #print(row)
        tokens = word_tokenize(str(row))
        #nltk library to extract POS tagging
        tagged=nltk.pos_tag(tokens)
        
        #print(tagged)
        if tagged:
            for t in tagged:
                #print(t)
                d=d+ ' '+t[1] +' '+t[0]
#                 print (d)
            taggedData.append(d)
        else:
            d="NAN"+ ' '+"NAN" +' '+"NAN"
            taggedData.append(d)
        #print(tagged)
    print(len(taggedData))
    return taggedData
    
    
    

In [ ]:
posData=POSBasedFeatures(testdata['text'])


In [ ]:
vectorizer = TfidfVectorizer(lowercase=False,ngram_range=(2,2))

#vectorizer = CountVectorizer()

X = vectorizer.fit_transform(posData)
print(X.shape)

featureNames=vectorizer.get_feature_names()



In [ ]:
# encode categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
 
labelencoder_Y = LabelEncoder()

#Y = labelencoder_Y.fit_transform(data.gender)
Y= labelencoder_Y.fit_transform(data.character)
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
#Training RF with TF_IDF + POS

clfRF_TF_POS = RandomForestClassifier(n_estimators=100, min_samples_split=4)
clfRF_TF_POS.fit(X_train, y_train) 
y_pred=clfRF_TF_POS.predict(X_test)
evaluate(y_test,y_pred,'Random Forest + POS + TF-IDF (Baseline-1) on splitted_test')


In [ ]:
#Training SVM with TF_IDF with PSO

clfSVM_TF_POS=SVC(gamma='auto',kernel='linear', C = 0.05)
clfSVM_TF_POS.fit(X_train, y_train) 
y_pred=clfSVM_TF_POS.predict(X_test)
evaluate(y_test,y_pred,'SVM + POS + TF-IDF (Baseline-2) on splitted_test')



In [ ]:
 #Test data

posData=POSBasedFeatures(testData['text'])

vectorizer = TfidfVectorizer(vocabulary=featureNames,ngram_range=(2,2))
tX = vectorizer.fit_transform(posData)
labelencoder_Y = LabelEncoder()

#Y = labelencoder_Y.fit_transform(data.gender)
y_Test= labelencoder_Y.fit_transform(testData.character)

In [ ]:
y_Pred=clfRF_TF_POS.predict(tX)
evaluate(y_Test,y_Pred,'Random Forest + POS + TF-IDF (Baseline-1) on Test Dataset') 

print(np.unique(y_Pred))

In [ ]:
y_Pred=clfSVM_TF_POS.predict(tX)
evaluate(y_Test,y_Pred,'SVM + POS + TF-IDF (Baseline-2) on Test Dataset')

print(np.unique(y_Pred))